In [43]:
import torch
import pandas as pd
import numpy as np
import torch.utils
import torchvision.transforms
from torch import nn
%matplotlib inline

In [44]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [45]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values,requires_grad=True,dtype=torch.float32)
X_ols_train = nn.functional.normalize(X_ols_train)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values,requires_grad=True,dtype=torch.float32)


#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20,shuffle=True)

In [46]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1],Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)
olsloss = nn.MSELoss()
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=1)

In [47]:
num_epochs = 10
olsnet.train()
for epoch in range(num_epochs):
    print('-------------------')
    print(f'{epoch+1}th epoch')
    print('===================')
    olstrainer.zero_grad()
    X, Y = next(iter(olsdataloader))
    output = olsnet(X)

    l = olsloss(output,Y)
    print(f'Loss ols:{l}')


    l.backward(retain_graph=True)

    olstrainer.step()
    print('Coefficients: ',olsnet[0].weight.data)


-------------------
1th epoch
Loss ols:34161504256.0
Coefficients:  tensor([[3.5048e+05, 2.3444e+02, 2.4253e+02, 4.8061e+03, 3.9146e+04]])
-------------------
2th epoch
Loss ols:289543192576.0
Coefficients:  tensor([[-6.9783e+05, -6.3762e+02, -6.0205e+02, -4.0791e+03, -1.0968e+05]])
-------------------
3th epoch
Loss ols:2540513525760.0
Coefficients:  tensor([[2.4565e+06, 1.9108e+03, 1.6622e+03, 2.8442e+04, 2.5876e+05]])
-------------------
4th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
5th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
6th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
7th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
8th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-------------------
9th epoch
Loss ols:nan
Coefficients:  tensor([[nan, nan, nan, nan, nan]])
-----------

In [48]:
#Loading in complex MNIST train and test datasets while transforming them to 64
trans = [torchvision.transforms.ToTensor()]
trans.insert(0, torchvision.transforms.Resize(64))
trans = torchvision.transforms.Compose(trans)


mnist_train = torchvision.datasets.FashionMNIST(
    root="../PyTorch_Basic/Datasets",train = True, transform=trans, download= True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../PyTorch_Basic/Datasets",train = False, transform=trans, download= True)

In [49]:
#Splitting our data into batch sizes
mnist_batch_size = 256

mnist_train_iter = torch.utils.data.DataLoader(mnist_train,mnist_batch_size,shuffle=True,num_workers=4)
mnist_test_iter = torch.utils.data.DataLoader(mnist_test,mnist_batch_size,shuffle=True,num_workers=4)

In [56]:
#Creating the model and initializing the weights
logit_net = nn.Sequential(nn.Flatten(),nn.Linear(784,10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

logit_net.apply(init_weights)

logit_loss = nn.CrossEntropyLoss(reduction='none')

logit_trainer = torch.optim.SGD(logit_net.parameters(), lr=0.1)

In [62]:
#Creating a trainer
def mnist_trainer(net,train_iter,loss,optim):


Parameter containing:
tensor([-0.0016,  0.0124,  0.0229,  0.0052, -0.0261, -0.0015, -0.0199, -0.0346,
         0.0112, -0.0335], requires_grad=True)